# 🚀 Price Forecast API Setup

This notebook:
1. Extracts unique brands and models from `expanded_prices_10k.xlsx`
2. Exports them to JSON for the React frontend
3. Creates a Flask API that uses the trained XGBoost model for 30-day forecasting


## Install Required Packages

Run this cell first to install all dependencies needed for the API.

In [27]:
!pip install fastapi uvicorn[standard] openpyxl -q

In [19]:
# Fix environment compatibility for FastAPI/Pydantic
!pip install -U typing_extensions pydantic annotated-types -q

In [23]:
# Force upgrade with explicit versions for compatibility
!python -m pip install -U typing_extensions==4.12.2 pydantic==2.10.4 fastapi==0.115.4 annotated-types==0.7.0 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.20.0 which is incompatible.


In [25]:
# Hard reset typing_extensions to a compatible version
!python -m pip uninstall -y typing_extensions
!python -m pip install typing_extensions==4.12.2 -q

import typing_extensions as te
print('typing_extensions path:', getattr(te, '__file__', 'unknown'))
print('Has Sentinel:', hasattr(te, 'Sentinel'))

Found existing installation: typing_extensions 4.12.2
Uninstalling typing_extensions-4.12.2:
  Successfully uninstalled typing_extensions-4.12.2
typing_extensions path: c:\Users\kvpra\OneDrive\Desktop\sample_project_1\myenv1\lib\site-packages\typing_extensions.py
Has Sentinel: False


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.20.0 which is incompatible.


In [26]:
# Try reloading typing_extensions in current kernel
import importlib, typing_extensions as te
te = importlib.reload(te)
print('After reload -> Has Sentinel:', hasattr(te, 'Sentinel'))

After reload -> Has Sentinel: False


## Step 1: Import Libraries and Load Data

In [28]:
import pandas as pd
import numpy as np
import json
import pickle
import xgboost as xgb
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load the Excel data
print("📊 Loading expanded_prices_10k.xlsx...")
df = pd.read_excel('expanded_prices_10k.xlsx')

print(f"✅ Loaded {len(df)} rows")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

📊 Loading expanded_prices_10k.xlsx...
✅ Loaded 11000 rows

Columns: ['product_id', 'Brand', 'Model', 'Category', 'Date', 'Time', 'SaleEvent', 'Price', 'DiscountPercentage']

First few rows:


,product_id,Brand,Model,Category,Date,Time,SaleEvent,Price,DiscountPercentage
0,P0004,OnePlus,Buds Z2,True Wireless Earbuds,2024-06-02,09:00:00,NaN,4999.00,0.00
1,P0004,OnePlus,Buds Z2,True Wireless Earbuds,2024-03-28,16:11:00,No Sale,5055.58,-1.13
2,P0006,Sennheiser,HD 450SE,On-Ear Headphones,2025-09-18,09:35:00,Republic Day Sale,8954.93,10.36
3,P0001,Bose,QuietComfort Ultra,On-Ear Headphones,2023-08-08,14:44:00,No Sale,39447.29,-1.41
4,P0004,OnePlus,Buds Z2,True Wireless Earbuds,2024-03-19,17:10:00,No Sale,5036.52,-0.75


In [29]:
# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Drop any rows with invalid dates (if any)
df = df.dropna(subset=['Date']).copy()
print('🗓️  Coerced Date column to datetime. Min:', df['Date'].min().date(), 'Max:', df['Date'].max().date())

🗓️  Coerced Date column to datetime. Min: 2023-01-01 Max: 2025-10-25


## Step 2: Extract Unique Brands and Models

In [30]:
# Get unique brands
brands = sorted(df['Brand'].unique().tolist())
print(f"📱 Found {len(brands)} unique brands:")
print(brands)

# Get models for each brand
models_by_brand = {}
for brand in brands:
    models = sorted(df[df['Brand'] == brand]['Model'].unique().tolist())
    models_by_brand[brand] = models
    print(f"\n{brand}: {len(models)} models")
    print(f"  Sample: {models[:3]}")

# Export to JSON for React frontend
brands_models_data = {
    'brands': brands,
    'modelsByBrand': models_by_brand
}

with open('brands_models.json', 'w') as f:
    json.dump(brands_models_data, f, indent=2)

print("\n✅ Exported brands_models.json for React frontend")

📱 Found 6 unique brands:
['Bose', 'JBL', 'OnePlus', 'Sennheiser', 'Sony', 'boAt']

Bose: 1 models
  Sample: ['QuietComfort Ultra']

JBL: 2 models
  Sample: ['Flip 6', 'Go 3']

OnePlus: 1 models
  Sample: ['Buds Z2']

Sennheiser: 2 models
  Sample: ['CX 80S', 'HD 450SE']

Sony: 3 models
  Sample: ['HT-S20R', 'WF-1000XM5', 'WH-CH520']

boAt: 3 models
  Sample: ['Aavante Bar 1160', 'Airdopes 141', 'Rockerz 450']

✅ Exported brands_models.json for React frontend


## Step 3: Load XGBoost Model and Artifacts

In [10]:
import os

required_files = [
    'xgboost_model_tuned.json',
    'label_encoders.pkl',
    'feature_columns.json'
]

print('🔍 Checking for required ML artifacts...')
missing = [f for f in required_files if not os.path.exists(f)]
for f in required_files:
    print(f" - {f}: {'✅ found' if f not in missing else '❌ missing'}")

if missing:
    print('\n❗ One or more required files are missing:')
    for f in missing:
        print(f"   • {f}")
    print('\nFix options:')
    print('  1) Open xgboost_model.ipynb and Run All to save artifacts into this same folder')
    print('  2) If you have them elsewhere, copy the files here')
    raise FileNotFoundError('Missing ML artifacts: ' + ', '.join(missing))

print('\n✅ All required artifacts are present. Proceeding to load the model...')

🔍 Checking for required ML artifacts...
 - xgboost_model_tuned.json: ✅ found
 - label_encoders.pkl: ✅ found
 - feature_columns.json: ✅ found

✅ All required artifacts are present. Proceeding to load the model...


In [31]:
# Load the trained XGBoost model
print("🤖 Loading XGBoost model...")
xgb_model = xgb.Booster()
xgb_model.load_model('xgboost_model_tuned.json')
print("✅ Model loaded successfully")

# Load label encoders
print("\n📦 Loading label encoders...")
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)
print(f"✅ Loaded {len(label_encoders)} label encoders")

# Load feature columns
print("\n📋 Loading feature columns...")
with open('feature_columns.json', 'r') as f:
    feature_cols = json.load(f)
print(f"✅ Loaded {len(feature_cols)} feature columns")

print("\n🎯 Model ready for forecasting!")

🤖 Loading XGBoost model...
✅ Model loaded successfully

📦 Loading label encoders...
✅ Loaded 6 label encoders

📋 Loading feature columns...
✅ Loaded 27 feature columns

🎯 Model ready for forecasting!


In [15]:
print('\n🔎 Feature columns (ordered):')
print(feature_cols)
print('\n🔑 Label encoders available:', list(label_encoders.keys()))


🔎 Feature columns (ordered):
['product_id', 'Brand', 'Model', 'Category', 'Time', 'SaleEvent', 'HasSaleEvent', 'year', 'month', 'day', 'dayofweek', 'quarter', 'week_of_year', 'is_weekend', 'is_month_start', 'is_month_end', 'days_in_month', 'DiscountPercentage_lag_1', 'DiscountPercentage_lag_2', 'DiscountPercentage_lag_3', 'DiscountPercentage_lag_7', 'DiscountPercentage_lag_14', 'DiscountPercentage_lag_30', 'DiscountPercentage_rolling_mean_7', 'DiscountPercentage_rolling_std_7', 'DiscountPercentage_rolling_mean_30', 'DiscountPercentage_rolling_std_30']

🔑 Label encoders available: ['Brand', 'Model', 'Category', 'product_id', 'SaleEvent', 'Time']


## Step 4: Create Forecast Function

This function will:
1. Filter data for specific brand and model
2. Get last 60 days of historical data
3. Use XGBoost model to predict next 30 days
4. Return both historical and forecast data

In [32]:
def generate_forecast(brand, model_name, days=30):
    """
    Generate price and discount forecast for a specific brand and model using the trained XGBoost model.

    Returns a dict with keys: brand, model, historical, forecast.
    - historical: last 60 days from the dataset with fields {date, price, discount}
    - forecast: next `days` days with fields {date, price, discount}
    """
    # Filter data for specific brand and model
    filtered_df = df[(df['Brand'] == brand) & (df['Model'] == model_name)].copy()
    if filtered_df.empty:
        return {"error": f"No data found for {brand} - {model_name}"}

    # Sort by date
    filtered_df = filtered_df.sort_values('Date').reset_index(drop=True)

    # Use most frequent categorical values for constant fields
    top_product_id = filtered_df['product_id'].mode().iat[0]
    top_category = filtered_df['Category'].mode().iat[0]
    top_time = filtered_df['Time'].mode().iat[0]

    # Prepare historical data (last 60 days)
    historical_data = filtered_df.tail(60)
    historical = [
        {
            'date': d.strftime('%Y-%m-%d'),
            'price': float(p),
            'discount': float(dc)
        }
        for d, p, dc in zip(historical_data['Date'], historical_data['Price'], historical_data['DiscountPercentage'])
    ]

    # For feature engineering of lags/rolling, maintain series of discounts
    discount_series = historical_data['DiscountPercentage'].tolist()

    # Helper to encode with fallback for unseen labels
    def encode_safe(le, value, fallback=None):
        classes = set(le.classes_.tolist()) if hasattr(le, 'classes_') else set()
        if value in classes:
            return int(le.transform([value])[0])
        fb = fallback if fallback is not None else list(classes)[0] if classes else value
        return int(le.transform([fb])[0])

    # Encoders
    enc_brand = label_encoders['Brand']
    enc_model = label_encoders['Model']
    enc_category = label_encoders['Category']
    enc_product = label_encoders['product_id']
    enc_time = label_encoders['Time']
    enc_sale = label_encoders['SaleEvent']

    # Encoded base values (with safe fallback)
    brand_encoded = encode_safe(enc_brand, brand)
    model_encoded = encode_safe(enc_model, model_name)
    category_encoded = encode_safe(enc_category, top_category)
    product_encoded = encode_safe(enc_product, top_product_id)
    time_encoded = encode_safe(enc_time, top_time)

    # Choose sale event for forecast
    sale_event_value = 'No Sale' if hasattr(enc_sale, 'classes_') and ('No Sale' in enc_sale.classes_) else (filtered_df['SaleEvent'].mode().iat[0] if not filtered_df['SaleEvent'].isna().all() else 'No Sale')
    sale_event_encoded = encode_safe(enc_sale, sale_event_value)

    # Price trend estimate (simple): mean pct change over last 14 days
    price_hist = historical_data['Price'].astype(float)
    recent_pct_change = price_hist.pct_change().tail(14).mean()
    if pd.isna(recent_pct_change):
        recent_pct_change = 0.0

    last_price = float(price_hist.iloc[-1])
    last_discount = float(historical_data['DiscountPercentage'].iloc[-1])

    forecast_data = []
    last_date = historical_data['Date'].max()

    for i in range(days):
        forecast_date = last_date + pd.Timedelta(days=i + 1)

        # Temporal features
        year = forecast_date.year
        month = forecast_date.month
        day = forecast_date.day
        dayofweek = forecast_date.dayofweek
        quarter = (month - 1) // 3 + 1
        week_of_year = int(forecast_date.strftime('%U'))
        is_weekend = 1 if dayofweek >= 5 else 0
        is_month_start = 1 if day == 1 else 0
        is_month_end = 1 if (forecast_date + pd.Timedelta(days=1)).month != month else 0
        days_in_month = int(pd.Period(forecast_date, freq='M').days_in_month)

        # Lag features for DiscountPercentage
        def lag(n):
            if len(discount_series) >= n:
                return float(discount_series[-n])
            return float(discount_series[-1])

        # Rolling stats helpers
        def rolling_mean(n):
            arr = discount_series[-n:] if len(discount_series) >= n else discount_series
            return float(pd.Series(arr).mean())

        def rolling_std(n):
            arr = discount_series[-n:] if len(discount_series) >= n else discount_series
            val = float(pd.Series(arr).std(ddof=0))
            return 0.0 if pd.isna(val) else val

        # Assemble feature row matching training columns exactly
        feat = {
            'product_id': product_encoded,
            'Brand': brand_encoded,
            'Model': model_encoded,
            'Category': category_encoded,
            'Time': time_encoded,
            'SaleEvent': sale_event_encoded,
            'HasSaleEvent': 0 if (isinstance(sale_event_value, str) and sale_event_value.lower() in ['no sale', 'nan']) else 1,
            'year': year,
            'month': month,
            'day': day,
            'dayofweek': dayofweek,
            'quarter': quarter,
            'week_of_year': week_of_year,
            'is_weekend': is_weekend,
            'is_month_start': is_month_start,
            'is_month_end': is_month_end,
            'days_in_month': days_in_month,
            'DiscountPercentage_lag_1': lag(1),
            'DiscountPercentage_lag_2': lag(2),
            'DiscountPercentage_lag_3': lag(3),
            'DiscountPercentage_lag_7': lag(7),
            'DiscountPercentage_lag_14': lag(14),
            'DiscountPercentage_lag_30': lag(30),
            'DiscountPercentage_rolling_mean_7': rolling_mean(7),
            'DiscountPercentage_rolling_std_7': rolling_std(7),
            'DiscountPercentage_rolling_mean_30': rolling_mean(30),
            'DiscountPercentage_rolling_std_30': rolling_std(30),
        }

        # Ensure correct column order and fill any missing required columns defensively
        row_df = pd.DataFrame([{col: feat.get(col, 0) for col in feature_cols}])

        # Predict discount using Booster
        dmat = xgb.DMatrix(row_df)
        pred_discount = float(xgb_model.predict(dmat)[0])

        # Clamp to reasonable range (-50% to 90%) to avoid outliers
        pred_discount = max(-50.0, min(90.0, pred_discount))

        # Update series for future lags/rolling
        discount_series.append(pred_discount)

        # Simple price projection: apply recent trend and adjust by discount delta
        discount_delta = (pred_discount - last_discount) / 100.0
        price_trend = recent_pct_change if not pd.isna(recent_pct_change) else 0.0
        adjusted = last_price * (1.0 + price_trend - 0.2 * discount_delta)
        last_price = float(max(1.0, adjusted))
        last_discount = pred_discount

        forecast_data.append({
            'date': forecast_date.strftime('%Y-%m-%d'),
            'price': round(last_price, 2),
            'discount': round(pred_discount, 2)
        })

    return {
        'brand': brand,
        'model': model_name,
        'historical': historical,
        'forecast': forecast_data
    }

# Test the function with first brand/model
test_brand = brands[0]
test_model = models_by_brand[test_brand][0]
print(f"🧪 Testing forecast for: {test_brand} - {test_model}")
result = generate_forecast(test_brand, test_model)
print(f"\n✅ Generated {len(result['historical'])} historical points")
print(f"✅ Generated {len(result['forecast'])} forecast points")

🧪 Testing forecast for: Bose - QuietComfort Ultra

✅ Generated 60 historical points
✅ Generated 30 forecast points


## Step 5: Create FastAPI Application

This API will serve forecast data to the React frontend. FastAPI provides:
- Automatic interactive documentation at `/docs`
- Better performance than Flask
- Built-in request/response validation

In [33]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Dict
import uvicorn

app = FastAPI(
    title="Price Forecast API",
    description="ML-powered price and discount forecasting using XGBoost",
    version="1.0.0"
)

# Enable CORS for React frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, replace with specific origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request/Response models
class ForecastRequest(BaseModel):
    brand: str
    model: str

class DataPoint(BaseModel):
    date: str
    price: float
    discount: float

class ForecastResponse(BaseModel):
    brand: str
    model: str
    historical: List[DataPoint]
    forecast: List[DataPoint]

class BrandsResponse(BaseModel):
    brands: List[str]
    modelsByBrand: Dict[str, List[str]]

@app.post("/api/forecast", response_model=ForecastResponse)
async def forecast(request: ForecastRequest):
    """
    Generate 30-day price and discount forecast for a specific brand and model.
    
    - **brand**: Brand name (e.g., "Apple")
    - **model**: Model name (e.g., "iPhone 13")
    """
    try:
        result = generate_forecast(request.brand, request.model)
        
        if 'error' in result:
            raise HTTPException(status_code=404, detail=result['error'])
        
        return result
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/brands", response_model=BrandsResponse)
async def get_brands():
    """Get all available brands and their models"""
    try:
        return {
            'brands': brands,
            'modelsByBrand': models_by_brand
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    """Health check endpoint"""
    return {"status": "ok", "message": "Forecast API is running"}

print("✅ FastAPI application created!")
print("\n📡 Available endpoints:")
print("   POST /api/forecast - Generate forecast for brand/model")
print("   GET  /api/brands - Get all brands and models")
print("   GET  /health - Health check")
print("   GET  /docs - Interactive API documentation")
print("   GET  /redoc - Alternative API documentation")

✅ FastAPI application created!

📡 Available endpoints:
   POST /api/forecast - Generate forecast for brand/model
   GET  /api/brands - Get all brands and models
   GET  /health - Health check
   GET  /docs - Interactive API documentation
   GET  /redoc - Alternative API documentation


In [24]:
import typing_extensions as te
import importlib, sys
print('typing_extensions path:', getattr(te, '__file__', 'unknown'))
print('typing_extensions version attr:', getattr(te, '__version__', 'unknown'))
print('Has Sentinel:', hasattr(te, 'Sentinel'))
print('Python:', sys.version)

typing_extensions path: c:\Users\kvpra\OneDrive\Desktop\sample_project_1\myenv1\lib\site-packages\typing_extensions.py
typing_extensions version attr: unknown
Has Sentinel: False
Python: 3.9.20 (main, Oct  3 2024, 07:38:01) [MSC v.1929 64 bit (AMD64)]


## Step 6: Run the FastAPI Server

Execute the cell below to start the FastAPI server on port 5000.

**Note:** 
- The server will run in this notebook and block further execution
- To stop it, interrupt the kernel
- Visit `http://localhost:5000/docs` for interactive API documentation!

## Step 6B: Alternative (Flask API Fallback)

If FastAPI import fails due to environment packages, you can run this Flask fallback without extra dependencies.

In [ ]:
try:
    from flask import Flask, request, jsonify
    from flask_cors import CORS
except ImportError:
    !python -m pip install flask flask-cors -q
    from flask import Flask, request, jsonify
    from flask_cors import CORS

flask_app = Flask(__name__)
CORS(flask_app)

@flask_app.route('/api/forecast', methods=['POST'])
def flask_forecast():
    data = request.json or {}
    brand = data.get('brand')
    model = data.get('model')
    if not brand or not model:
        return jsonify({'error': 'Brand and model are required'}), 400
    res = generate_forecast(brand, model)
    if 'error' in res:
        return jsonify(res), 404
    return jsonify(res), 200

@flask_app.route('/api/brands', methods=['GET'])
def flask_brands():
    return jsonify({'brands': brands, 'modelsByBrand': models_by_brand}), 200

@flask_app.route('/health', methods=['GET'])
def flask_health():
    return jsonify({'status': 'ok', 'framework': 'flask'}), 200

print('✅ Flask fallback API is ready.')

In [ ]:
# Run the Flask fallback server
if __name__ == '__main__':
    print('🚀 Starting Flask fallback on http://localhost:5000 ...')
    flask_app.run(host='0.0.0.0', port=5000, debug=True)

In [34]:
if __name__ == '__main__':
    print("🚀 Starting FastAPI server...")
    print("📍 Server will be available at: http://localhost:5000")
    print("📚 API Documentation at: http://localhost:5000/docs")
    print("⚠️  To stop the server, interrupt the kernel (Ctrl+C or Interrupt button)")
    print("\n" + "="*60 + "\n")
    uvicorn.run(app, host="0.0.0.0", port=5000)

🚀 Starting FastAPI server...
📍 Server will be available at: http://localhost:5000
📚 API Documentation at: http://localhost:5000/docs
⚠️  To stop the server, interrupt the kernel (Ctrl+C or Interrupt button)




RuntimeError: asyncio.run() cannot be called from a running event loop